In [1]:
%config IPCompleter.greedy=True
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, re, math, datetime as dt, pandas as pd, numpy as np, time
import logging
import matplotlib.pyplot as plt
from string import Template
from IPython.display import display, HTML

logging.basicConfig(format='%(asctime)s [%(name)s:%(lineno)d:%(funcName)s] [%(levelname)s] %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.width', 5000)

def display_df(df):
    display(df.head(4))
    print(df.shape)

In [3]:
import fastf1 as f1

session = f1.get_session(2023, 8, 'R')
schedule = f1.get_event_schedule(2023)
# print(session.event)
display_df(schedule)
schedule.dtypes

req         WARNING 	

DEFAULT CACHE ENABLED!
	Cache directory: /Users/hannahwang/Library/Caches/fastf1.
	Size: 298.88 MB
2023-12-29 15:44:59,007 [fastf1.fastf1.req:521:_enable_default_cache] [WARNING] 

DEFAULT CACHE ENABLED!
	Cache directory: /Users/hannahwang/Library/Caches/fastf1.
	Size: 298.88 MB


,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,Session2,Session2Date,Session2DateUtc,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023-02-25,Pre-Season Testing,testing,Practice 1,2023-02-23 10:00:00+03:00,2023-02-23 07:00:00,Practice 2,2023-02-24 10:00:00+03:00,2023-02-24 07:00:00,Practice 3,2023-02-25 10:00:00+03:00,2023-02-25 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023-03-05,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 14:30:00+03:00,2023-03-03 11:30:00,Practice 2,2023-03-03 18:00:00+03:00,2023-03-03 15:00:00,Practice 3,2023-03-04 14:30:00+03:00,2023-03-04 11:30:00,Qualifying,2023-03-04 18:00:00+03:00,2023-03-04 15:00:00,Race,2023-03-05 18:00:00+03:00,2023-03-05 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023-03-19,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 16:30:00+03:00,2023-03-17 13:30:00,Practice 2,2023-03-17 20:00:00+03:00,2023-03-17 17:00:00,Practice 3,2023-03-18 16:30:00+03:00,2023-03-18 13:30:00,Qualifying,2023-03-18 20:00:00+03:00,2023-03-18 17:00:00,Race,2023-03-19 20:00:00+03:00,2023-03-19 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023-04-02,Australian Grand Prix,conventional,Practice 1,2023-03-31 12:30:00+10:00,2023-03-31 02:30:00,Practice 2,2023-03-31 16:00:00+10:00,2023-03-31 06:00:00,Practice 3,2023-04-01 12:30:00+10:00,2023-04-01 02:30:00,Qualifying,2023-04-01 16:00:00+10:00,2023-04-01 06:00:00,Race,2023-04-02 15:00:00+10:00,2023-04-02 05:00:00,True


(23, 23)


RoundNumber                   int64
Country                      object
Location                     object
OfficialEventName            object
EventDate            datetime64[ns]
EventName                    object
EventFormat                  object
Session1                     object
Session1Date                 object
Session1DateUtc      datetime64[ns]
Session2                     object
Session2Date                 object
Session2DateUtc      datetime64[ns]
Session3                     object
Session3Date                 object
Session3DateUtc      datetime64[ns]
Session4                     object
Session4Date                 object
Session4DateUtc      datetime64[ns]
Session5                     object
Session5Date                 object
Session5DateUtc      datetime64[ns]
F1ApiSupport                   bool
dtype: object

In [4]:
session.load()
session.results

core           INFO 	Loading data for Canadian Grand Prix - Race [v3.1.6]
2023-12-29 15:44:59,058 [fastf1.fastf1.core:1248:load] [INFO] Loading data for Canadian Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
2023-12-29 15:44:59,060 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_info
req            INFO 	Using cached data for driver_info
2023-12-29 15:44:59,061 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
2023-12-29 15:45:01,941 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2023-12-29 15:45:01,943 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2023-12-29 15:45:01,945 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png,,1.0,1,1.0,NaT,NaT,NaT,0 days 01:33:58.348000,Finished,25.0
14,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png,,2.0,2,2.0,NaT,NaT,NaT,0 days 00:00:09.570000,Finished,18.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,6CD3BF,mercedes,Lewis,Hamilton,Lewis Hamilton,https://www.formula1.com/content/dam/fom-website/drivers/L/LEWHAM01_Lewis_Hamilton/lewham01.png.transform/1col/image.png,,3.0,3,3.0,NaT,NaT,NaT,0 days 00:00:04.598000,Finished,15.0
16,16,C LECLERC,LEC,leclerc,Ferrari,F91536,ferrari,Charles,Leclerc,Charles Leclerc,https://www.formula1.com/content/dam/fom-website/drivers/C/CHALEC01_Charles_Leclerc/chalec01.png.transform/1col/image.png,,4.0,4,10.0,NaT,NaT,NaT,0 days 00:00:18.648000,Finished,12.0
55,55,C SAINZ,SAI,sainz,Ferrari,F91536,ferrari,Carlos,Sainz,Carlos Sainz,https://www.formula1.com/content/dam/fom-website/drivers/C/CARSAI01_Carlos_Sainz/carsai01.png.transform/1col/image.png,,5.0,5,11.0,NaT,NaT,NaT,0 days 00:00:21.540000,Finished,10.0
11,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,https://www.formula1.com/content/dam/fom-website/drivers/S/SERPER01_Sergio_Perez/serper01.png.transform/1col/image.png,,6.0,6,12.0,NaT,NaT,NaT,0 days 00:00:51.028000,Finished,9.0
23,23,A ALBON,ALB,albon,Williams,37BEDD,williams,Alexander,Albon,Alexander Albon,https://www.formula1.com/content/dam/fom-website/drivers/A/ALEALB01_Alexander_Albon/alealb01.png.transform/1col/image.png,,7.0,7,9.0,NaT,NaT,NaT,0 days 00:01:00.813000,Finished,6.0
31,31,E OCON,OCO,ocon,Alpine,2293D1,alpine,Esteban,Ocon,Esteban Ocon,https://www.formula1.com/content/dam/fom-website/drivers/E/ESTOCO01_Esteban_Ocon/estoco01.png.transform/1col/image.png,,8.0,8,6.0,NaT,NaT,NaT,0 days 00:01:01.692000,Finished,4.0
18,18,L STROLL,STR,stroll,Aston Martin,358C75,aston_martin,Lance,Stroll,Lance Stroll,https://www.formula1.com/content/dam/fom-website/drivers/L/LANSTR01_Lance_Stroll/lanstr01.png.transform/1col/image.png,,9.0,9,16.0,NaT,NaT,NaT,0 days 00:01:04.402000,Finished,2.0
77,77,V BOTTAS,BOT,bottas,Alfa Romeo,C92D4B,alfa,Valtteri,Bottas,Valtteri Bottas,https://www.formula1.com/content/dam/fom-website/drivers/V/VALBOT01_Valtteri_Bottas/valbot01.png.transform/1col/image.png,,10.0,10,14.0,NaT,NaT,NaT,0 days 00:01:04.432000,Finished,1.0


In [5]:
session.laps.head(25)

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:03:28.836000,VER,1,0 days 00:01:23.080000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:25.105000,0 days 00:00:31.257000,NaT,0 days 01:02:57.649000,0 days 01:03:28.914000,250.0,267.0,274.0,300.0,False,MEDIUM,1.0,True,Red Bull Racing,0 days 01:02:05.478000,2023-06-18 18:03:05.484,1,1.0,False,,False,False
1,0 days 01:04:46.911000,VER,1,0 days 00:01:18.075000,2.0,1.0,NaT,NaT,0 days 00:00:21.686000,0 days 00:00:25.051000,0 days 00:00:31.338000,0 days 01:03:50.522000,0 days 01:04:15.573000,0 days 01:04:46.911000,243.0,270.0,275.0,301.0,True,MEDIUM,2.0,True,Red Bull Racing,0 days 01:03:28.836000,2023-06-18 18:04:28.842,1,1.0,False,,False,True
2,0 days 01:06:04.678000,VER,1,0 days 00:01:17.767000,3.0,1.0,NaT,NaT,0 days 00:00:21.626000,0 days 00:00:24.909000,0 days 00:00:31.232000,0 days 01:05:08.537000,0 days 01:05:33.446000,0 days 01:06:04.678000,246.0,271.0,276.0,300.0,True,MEDIUM,3.0,True,Red Bull Racing,0 days 01:04:46.911000,2023-06-18 18:05:46.917,1,1.0,False,,False,True
3,0 days 01:07:21.881000,VER,1,0 days 00:01:17.203000,4.0,1.0,NaT,NaT,0 days 00:00:21.438000,0 days 00:00:24.716000,0 days 00:00:31.049000,0 days 01:06:26.116000,0 days 01:06:50.832000,0 days 01:07:21.881000,250.0,269.0,276.0,301.0,True,MEDIUM,4.0,True,Red Bull Racing,0 days 01:06:04.678000,2023-06-18 18:07:04.684,1,1.0,False,,False,True
4,0 days 01:08:39.252000,VER,1,0 days 00:01:17.371000,5.0,1.0,NaT,NaT,0 days 00:00:21.493000,0 days 00:00:24.718000,0 days 00:00:31.160000,0 days 01:07:43.374000,0 days 01:08:08.092000,0 days 01:08:39.252000,NaN,268.0,275.0,302.0,False,MEDIUM,5.0,True,Red Bull Racing,0 days 01:07:21.881000,2023-06-18 18:08:21.887,1,1.0,False,,False,True
5,0 days 01:09:56.859000,VER,1,0 days 00:01:17.607000,6.0,1.0,NaT,NaT,0 days 00:00:21.609000,0 days 00:00:24.763000,0 days 00:00:31.235000,0 days 01:09:00.861000,0 days 01:09:25.624000,0 days 01:09:56.859000,239.0,272.0,275.0,301.0,False,MEDIUM,6.0,True,Red Bull Racing,0 days 01:08:39.252000,2023-06-18 18:09:39.258,1,1.0,False,,False,True
6,0 days 01:11:14.424000,VER,1,0 days 00:01:17.565000,7.0,1.0,NaT,NaT,0 days 00:00:21.600000,0 days 00:00:24.770000,0 days 00:00:31.195000,0 days 01:10:18.459000,0 days 01:10:43.229000,0 days 01:11:14.424000,238.0,272.0,276.0,300.0,False,MEDIUM,7.0,True,Red Bull Racing,0 days 01:09:56.859000,2023-06-18 18:10:56.865,2,1.0,False,,False,True
7,0 days 01:12:47.079000,VER,1,0 days 00:01:32.655000,8.0,1.0,NaT,NaT,0 days 00:00:28.067000,0 days 00:00:33.169000,0 days 00:00:31.419000,0 days 01:11:42.491000,0 days 01:12:15.660000,0 days 01:12:47.079000,172.0,229.0,275.0,302.0,False,MEDIUM,8.0,True,Red Bull Racing,0 days 01:11:14.424000,2023-06-18 18:12:14.430,267,1.0,False,,False,False
8,0 days 01:14:04.574000,VER,1,0 days 00:01:17.495000,9.0,1.0,NaT,NaT,0 days 00:00:21.715000,0 days 00:00:24.787000,0 days 00:00:30.993000,0 days 01:13:08.794000,0 days 01:13:33.581000,0 days 01:14:04.574000,238.0,269.0,274.0,303.0,False,MEDIUM,9.0,True,Red Bull Racing,0 days 01:12:47.079000,2023-06-18 18:13:47.085,1,1.0,False,,False,True
9,0 days 01:15:21.933000,VER,1,0 days 00:01:17.359000,10.0,1.0,NaT,NaT,0 days 00:00:21.577000,0 days 00:00:24.746000,0 days 00:00:31.036000,0 days 01:14:26.151000,0 days 01:14:50.897000,0 days 01:15:21.933000,239.0,269.0,276.0,302.0,False,MEDIUM,10.0,True,Red Bull Racing,0 days 01:14:04.574000,2023-06-18 18:15:04.580,1,1.0,False,,False,True


In [6]:
session.laps.pick_laps(5).pick_driver('VER').get_car_data()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2023-06-18 18:08:22.065,11150,281,7,100,False,0,car,0 days 00:00:00.178000,0 days 01:07:22.059000
1,2023-06-18 18:08:22.265,11253,282,7,100,False,0,car,0 days 00:00:00.378000,0 days 01:07:22.259000
2,2023-06-18 18:08:22.745,11237,283,7,100,False,0,car,0 days 00:00:00.858000,0 days 01:07:22.739000
3,2023-06-18 18:08:22.985,11206,284,7,100,False,0,car,0 days 00:00:01.098000,0 days 01:07:22.979000
4,2023-06-18 18:08:23.185,11012,282,7,0,False,0,car,0 days 00:00:01.298000,0 days 01:07:23.179000
5,2023-06-18 18:08:23.505,10678,275,7,0,False,0,car,0 days 00:00:01.618000,0 days 01:07:23.499000
6,2023-06-18 18:08:23.745,10272,268,7,0,True,0,car,0 days 00:00:01.858000,0 days 01:07:23.739000
7,2023-06-18 18:08:23.945,9773,249,7,0,True,0,car,0 days 00:00:02.058000,0 days 01:07:23.939000
8,2023-06-18 18:08:24.345,10367,202,6,0,True,0,car,0 days 00:00:02.458000,0 days 01:07:24.339000
9,2023-06-18 18:08:24.585,10265,176,6,0,True,0,car,0 days 00:00:02.698000,0 days 01:07:24.579000


In [7]:
# session.race_control_messages()

In [8]:
display(session.get_circuit_info().corners)

,X,Y,Number,Letter,Angle,Distance
0,3314.213623,-1317.854858,1,,-165.704223,225.743213
1,3856.566650,-2073.488770,2,,-22.292629,320.624280
2,740.000488,-608.991943,3,,-158.882111,693.206801
3,746.362305,-42.993057,4,,10.981968,750.568160
4,-724.044739,1618.967407,5,,29.699708,967.507804
5,-993.916626,3790.888672,6,,27.681895,1198.245481
6,-1783.871460,4016.300781,7,,-117.805015,1292.609426
7,-1785.440430,10539.706055,8,,118.593733,1972.048065
8,-1140.291992,10976.314453,9,,-28.305198,2047.220648
9,-711.505371,16759.564453,10,,97.796495,2643.216507


In [9]:
session.laps.pick_driver("VER").pick_laps([19]).get_car_data()[:10]

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2023-06-18 18:30:30.389,11111,281,7,100,False,1,car,0 days 00:00:00.031000,0 days 01:29:30.383000
1,2023-06-18 18:30:30.669,11203,283,7,100,False,1,car,0 days 00:00:00.311000,0 days 01:29:30.663000
2,2023-06-18 18:30:30.869,11248,284,7,100,False,1,car,0 days 00:00:00.511000,0 days 01:29:30.863000
3,2023-06-18 18:30:31.189,11292,284,7,100,False,1,car,0 days 00:00:00.831000,0 days 01:29:31.183000
4,2023-06-18 18:30:31.549,11250,284,7,100,False,1,car,0 days 00:00:01.191000,0 days 01:29:31.543000
5,2023-06-18 18:30:31.789,11204,284,7,76,False,1,car,0 days 00:00:01.431000,0 days 01:29:31.783000
6,2023-06-18 18:30:31.949,10862,280,7,0,False,1,car,0 days 00:00:01.591000,0 days 01:29:31.943000
7,2023-06-18 18:30:32.349,9761,261,7,0,True,1,car,0 days 00:00:01.991000,0 days 01:29:32.343000
8,2023-06-18 18:30:32.709,9794,217,7,0,True,1,car,0 days 00:00:02.351000,0 days 01:29:32.703000
9,2023-06-18 18:30:33.029,10699,198,6,0,True,0,car,0 days 00:00:02.671000,0 days 01:29:33.023000


In [10]:
session.laps.pick_driver("VER").pick_laps([19]).get_pos_data()

,Date,Status,X,Y,Z,Source,Time,SessionTime
0,2023-06-18 18:30:30.365,OnTrack,3349,947,129,pos,0 days 00:00:00.007000,0 days 01:29:30.359000
1,2023-06-18 18:30:30.585,OnTrack,3372,777,128,pos,0 days 00:00:00.227000,0 days 01:29:30.579000
2,2023-06-18 18:30:30.965,OnTrack,3393,478,128,pos,0 days 00:00:00.607000,0 days 01:29:30.959000
3,2023-06-18 18:30:31.145,OnTrack,3395,273,128,pos,0 days 00:00:00.787000,0 days 01:29:31.139000
4,2023-06-18 18:30:31.565,OnTrack,3378,-42,129,pos,0 days 00:00:01.207000,0 days 01:29:31.559000
5,2023-06-18 18:30:31.804,OnTrack,3365,-183,130,pos,0 days 00:00:01.446000,0 days 01:29:31.798000
6,2023-06-18 18:30:32.124,OnTrack,3338,-429,130,pos,0 days 00:00:01.766000,0 days 01:29:32.118000
7,2023-06-18 18:30:32.324,OnTrack,3320,-574,131,pos,0 days 00:00:01.966000,0 days 01:29:32.318000
8,2023-06-18 18:30:32.584,OnTrack,3295,-805,132,pos,0 days 00:00:02.226000,0 days 01:29:32.578000
9,2023-06-18 18:30:33.004,OnTrack,3282,-989,133,pos,0 days 00:00:02.646000,0 days 01:29:32.998000


In [11]:
session.laps.shape

(1317, 31)

In [12]:
session.laps.pick_driver("VER").pick_laps([1,70]).get_car_data()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime
0,2023-06-18 18:03:05.824,9515,0,1,16,False,1,car,0 days 00:00:00.340000,0 days 01:02:05.818000
1,2023-06-18 18:03:06.264,6435,4,1,16,False,1,car,0 days 00:00:00.780000,0 days 01:02:06.258000
2,2023-06-18 18:03:06.424,5315,14,1,16,False,1,car,0 days 00:00:00.940000,0 days 01:02:06.418000
3,2023-06-18 18:03:06.624,4203,24,1,16,False,1,car,0 days 00:00:01.140000,0 days 01:02:06.618000
4,2023-06-18 18:03:06.904,4457,30,1,23,False,1,car,0 days 00:00:01.420000,0 days 01:02:06.898000
...,...,...,...,...,...,...,...,...,...,...
21241,2023-06-18 19:37:02.643,10706,270,7,100,False,0,car,0 days 01:33:57.159000,0 days 02:36:02.637000
21242,2023-06-18 19:37:02.883,10834,273,7,100,False,0,car,0 days 01:33:57.399000,0 days 02:36:02.877000
21243,2023-06-18 19:37:03.083,10846,275,7,100,False,0,car,0 days 01:33:57.599000,0 days 02:36:03.077000
21244,2023-06-18 19:37:03.443,10459,274,7,0,False,0,car,0 days 01:33:57.959000,0 days 02:36:03.437000


In [13]:
session.total_laps

70

AttributeError: 'Session' object has no attribute 'get_results'

In [ ]:
session.get_driver("VER")

In [15]:
session_results = session.results

In [16]:
session_results.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png,,1.0,1,1.0,NaT,NaT,NaT,0 days 01:33:58.348000,Finished,25.0
14,14,F ALONSO,ALO,alonso,Aston Martin,358C75,aston_martin,Fernando,Alonso,Fernando Alonso,https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png,,2.0,2,2.0,NaT,NaT,NaT,0 days 00:00:09.570000,Finished,18.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,6CD3BF,mercedes,Lewis,Hamilton,Lewis Hamilton,https://www.formula1.com/content/dam/fom-website/drivers/L/LEWHAM01_Lewis_Hamilton/lewham01.png.transform/1col/image.png,,3.0,3,3.0,NaT,NaT,NaT,0 days 00:00:04.598000,Finished,15.0
16,16,C LECLERC,LEC,leclerc,Ferrari,F91536,ferrari,Charles,Leclerc,Charles Leclerc,https://www.formula1.com/content/dam/fom-website/drivers/C/CHALEC01_Charles_Leclerc/chalec01.png.transform/1col/image.png,,4.0,4,10.0,NaT,NaT,NaT,0 days 00:00:18.648000,Finished,12.0
55,55,C SAINZ,SAI,sainz,Ferrari,F91536,ferrari,Carlos,Sainz,Carlos Sainz,https://www.formula1.com/content/dam/fom-website/drivers/C/CARSAI01_Carlos_Sainz/carsai01.png.transform/1col/image.png,,5.0,5,11.0,NaT,NaT,NaT,0 days 00:00:21.540000,Finished,10.0


In [56]:
session.get_driver("ALB")

DriverNumber                                                                                                                                 23
BroadcastName                                                                                                                           A ALBON
Abbreviation                                                                                                                                ALB
DriverId                                                                                                                                  albon
TeamName                                                                                                                               Williams
TeamColor                                                                                                                                37BEDD
TeamId                                                                                                                                 w

In [19]:
from fastf1.ergast import Ergast
ergast = Ergast()

In [24]:
ergast.get_driver_info(season=2023)

,driverId,driverNumber,driverCode,driverUrl,givenName,familyName,dateOfBirth,driverNationality
0,albon,23,ALB,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander,Albon,1996-03-23,Thai
1,alonso,14,ALO,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando,Alonso,1981-07-29,Spanish
2,bottas,77,BOT,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri,Bottas,1989-08-28,Finnish
3,de_vries,21,DEV,http://en.wikipedia.org/wiki/Nyck_de_Vries,Nyck,de Vries,1995-02-06,Dutch
4,gasly,10,GAS,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre,Gasly,1996-02-07,French
5,hamilton,44,HAM,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis,Hamilton,1985-01-07,British
6,hulkenberg,27,HUL,http://en.wikipedia.org/wiki/Nico_H%C3%BClkenberg,Nico,Hülkenberg,1987-08-19,German
7,lawson,40,LAW,http://en.wikipedia.org/wiki/Liam_Lawson,Liam,Lawson,2002-02-11,New Zealander
8,leclerc,16,LEC,http://en.wikipedia.org/wiki/Charles_Leclerc,Charles,Leclerc,1997-10-16,Monegasque
9,kevin_magnussen,20,MAG,http://en.wikipedia.org/wiki/Kevin_Magnussen,Kevin,Magnussen,1992-10-05,Danish


In [26]:
ergast.get_seasons(2023)

""


In [27]:
ergast.get_constructor_info(2023)

,constructorId,constructorUrl,constructorName,constructorNationality
0,alfa,http://en.wikipedia.org/wiki/Alfa_Romeo_in_Formula_One,Alfa Romeo,Swiss
1,alphatauri,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,AlphaTauri,Italian
2,alpine,http://en.wikipedia.org/wiki/Alpine_F1_Team,Alpine F1 Team,French
3,aston_martin,http://en.wikipedia.org/wiki/Aston_Martin_in_Formula_One,Aston Martin,British
4,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian
5,haas,http://en.wikipedia.org/wiki/Haas_F1_Team,Haas F1 Team,American
6,mclaren,http://en.wikipedia.org/wiki/McLaren,McLaren,British
7,mercedes,http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One,Mercedes,German
8,red_bull,http://en.wikipedia.org/wiki/Red_Bull_Racing,Red Bull,Austrian
9,williams,http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering,Williams,British


In [28]:
ergast.get_finishing_status(2023)

,statusId,count,status
0,1,304,Finished
1,2,2,Disqualified
2,3,9,Accident
3,4,9,Collision
4,5,10,Engine
5,6,2,Gearbox
6,10,1,Electrical
7,11,64,+1 Lap
8,12,6,+2 Laps
9,21,1,Radiator


In [41]:
ergast.get_driver_standings(2019).content[0]

,position,positionText,points,wins,driverId,driverNumber,driverCode,driverUrl,givenName,familyName,dateOfBirth,driverNationality,constructorIds,constructorUrls,constructorNames,constructorNationalities
0,1,1,413.0,11,hamilton,44,HAM,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis,Hamilton,1985-01-07,British,[mercedes],[http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One],[Mercedes],[German]
1,2,2,326.0,4,bottas,77,BOT,http://en.wikipedia.org/wiki/Valtteri_Bottas,Valtteri,Bottas,1989-08-28,Finnish,[mercedes],[http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One],[Mercedes],[German]
2,3,3,278.0,3,max_verstappen,33,VER,http://en.wikipedia.org/wiki/Max_Verstappen,Max,Verstappen,1997-09-30,Dutch,[red_bull],[http://en.wikipedia.org/wiki/Red_Bull_Racing],[Red Bull],[Austrian]
3,4,4,264.0,2,leclerc,16,LEC,http://en.wikipedia.org/wiki/Charles_Leclerc,Charles,Leclerc,1997-10-16,Monegasque,[ferrari],[http://en.wikipedia.org/wiki/Scuderia_Ferrari],[Ferrari],[Italian]
4,5,5,240.0,1,vettel,5,VET,http://en.wikipedia.org/wiki/Sebastian_Vettel,Sebastian,Vettel,1987-07-03,German,[ferrari],[http://en.wikipedia.org/wiki/Scuderia_Ferrari],[Ferrari],[Italian]
5,6,6,96.0,0,sainz,55,SAI,http://en.wikipedia.org/wiki/Carlos_Sainz_Jr.,Carlos,Sainz,1994-09-01,Spanish,[mclaren],[http://en.wikipedia.org/wiki/McLaren],[McLaren],[British]
6,7,7,95.0,0,gasly,10,GAS,http://en.wikipedia.org/wiki/Pierre_Gasly,Pierre,Gasly,1996-02-07,French,"[red_bull, toro_rosso]","[http://en.wikipedia.org/wiki/Red_Bull_Racing, http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso]","[Red Bull, Toro Rosso]","[Austrian, Italian]"
7,8,8,92.0,0,albon,23,ALB,http://en.wikipedia.org/wiki/Alexander_Albon,Alexander,Albon,1996-03-23,Thai,"[toro_rosso, red_bull]","[http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso, http://en.wikipedia.org/wiki/Red_Bull_Racing]","[Toro Rosso, Red Bull]","[Italian, Austrian]"
8,9,9,54.0,0,ricciardo,3,RIC,http://en.wikipedia.org/wiki/Daniel_Ricciardo,Daniel,Ricciardo,1989-07-01,Australian,[renault],[http://en.wikipedia.org/wiki/Renault_in_Formula_One],[Renault],[French]
9,10,10,52.0,0,perez,11,PER,http://en.wikipedia.org/wiki/Sergio_P%C3%A9rez,Sergio,Pérez,1990-01-26,Mexican,[racing_point],[http://en.wikipedia.org/wiki/Racing_Point_F1_Team],[Racing Point],[British]


In [42]:
session.name

'Race'

In [52]:
session.event.EventDate.month

6

In [53]:
session.total_laps

70

In [57]:
session.total_distance

AttributeError: 'Session' object has no attribute 'total_distance'

In [59]:
circuit_df = ergast.get_circuits(2022)

In [64]:
name_list = circuit_df.circuitName.tolist()
id_list = circuit_df.circuitId.tolist()

In [65]:
circuit_df

,circuitId,circuitUrl,circuitName,lat,long,locality,country
0,albert_park,http://en.wikipedia.org/wiki/Melbourne_Grand_Prix_Circuit,Albert Park Grand Prix Circuit,-37.8497,144.96800,Melbourne,Australia
1,americas,http://en.wikipedia.org/wiki/Circuit_of_the_Americas,Circuit of the Americas,30.1328,-97.64110,Austin,USA
2,bahrain,http://en.wikipedia.org/wiki/Bahrain_International_Circuit,Bahrain International Circuit,26.0325,50.51060,Sakhir,Bahrain
3,baku,http://en.wikipedia.org/wiki/Baku_City_Circuit,Baku City Circuit,40.3725,49.85330,Baku,Azerbaijan
4,catalunya,http://en.wikipedia.org/wiki/Circuit_de_Barcelona-Catalunya,Circuit de Barcelona-Catalunya,41.5700,2.26111,Montmeló,Spain
5,hungaroring,http://en.wikipedia.org/wiki/Hungaroring,Hungaroring,47.5789,19.24860,Budapest,Hungary
6,imola,http://en.wikipedia.org/wiki/Autodromo_Enzo_e_Dino_Ferrari,Autodromo Enzo e Dino Ferrari,44.3439,11.71670,Imola,Italy
7,interlagos,http://en.wikipedia.org/wiki/Aut%C3%B3dromo_Jos%C3%A9_Carlos_Pace,Autódromo José Carlos Pace,-23.7036,-46.69970,São Paulo,Brazil
8,jeddah,http://en.wikipedia.org/wiki/Jeddah_Street_Circuit,Jeddah Corniche Circuit,21.6319,39.10440,Jeddah,Saudi Arabia
9,marina_bay,http://en.wikipedia.org/wiki/Marina_Bay_Street_Circuit,Marina Bay Street Circuit,1.2914,103.86400,Marina Bay,Singapore


In [67]:
new_session = f1.get_session(2023, 'Albert Park Grand Prix Circuit', 'R')

In [68]:
new_session.event

RoundNumber                                                    14
Country                                                     Italy
Location                                                    Monza
OfficialEventName    FORMULA 1 PIRELLI GRAN PREMIO D’ITALIA 2023 
EventDate                                     2023-09-03 00:00:00
EventName                                      Italian Grand Prix
EventFormat                                          conventional
Session1                                               Practice 1
Session1Date                            2023-09-01 13:30:00+02:00
Session1DateUtc                               2023-09-01 11:30:00
Session2                                               Practice 2
Session2Date                            2023-09-01 17:00:00+02:00
Session2DateUtc                               2023-09-01 15:00:00
Session3                                               Practice 3
Session3Date                            2023-09-02 12:30:00+02:00
Session3Da

In [ ]:
races = pd.read_csv("../f1_data/race_data.csv")

df = races[(races['raceId'].isin(raceIds)) & (races.year==year)]
 sorted_df = df.sort_values('date', ascending=False)
 print(sorted_df)
 print(list(set(sorted_df['circuitId'])))